## References
* [Regular Expressions Documentation - Groups](https://docs.python.org/3/library/re.html#index-17)
* [Pandas documentation - pandas.Series.replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.replace.html?highlight=replace#pandas.Series.replace)
* [Pandas documentation - convert to categorical dtype](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html?highlight=string#series-creation)

# Data Manipulation Pandas - Jeopardy Project
## Project Goals

You will work to write several functions that investigate a dataset of 'Jeopardy!' questions and answers. Filter the dataset for topics that you’re interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion.

## Project Requirements
We’ve provided a csv file containing data about the game show 'Jeopardy!' in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

Note that in order to make this project as "real-world" as possible, we haven’t modified the data at all — we’re giving it to you exactly how we found it. As a result, this data isn’t as "clean" as the datasets you normally find on Codecademy. More specifically, there’s something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier the rest of the project.

## Load and Clean the dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
jeopardy_data = pd.read_csv('./materials/jeopardy.csv',
                            parse_dates = [' Air Date']
                           )
jeopardy_data.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [3]:
jeopardy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Show Number  216930 non-null  int64         
 1    Air Date    216930 non-null  datetime64[ns]
 2    Round       216930 non-null  object        
 3    Category    216930 non-null  object        
 4    Value       216930 non-null  object        
 5    Question    216930 non-null  object        
 6    Answer      216928 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 11.6+ MB


### Clean column names
There is a space in front of the names, remove and lower case.

In [4]:
jeopardy_data.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [5]:
jeopardy_data.rename(columns = {'Show Number': 'show_num',
                               ' Air Date': 'air_date',
                               ' Round': 'rounds',
                               ' Category': 'category',
                               ' Value': 'price',
                               ' Question': 'question',
                               ' Answer': 'answer'
                              }, inplace = True)

### Check for duplicates

In [6]:
jeopardy_data.duplicated().value_counts()

False    216930
dtype: int64

In [7]:
jeopardy_data.question.duplicated().value_counts()

False    216124
True        806
Name: question, dtype: int64

In [8]:
jeopardy_data[(jeopardy_data.question.duplicated()) & (jeopardy_data.category == 'NOTHING')]

,show_num,air_date,rounds,category,price,question,answer


### Clean `price` column
+ Remove `$` and `,` and convert to numeric (int).


+ Check for missing values: replace `'None'` with `0` (I am considering `'None'` as zero dollars, and not as `NaN`)

In [8]:
jeopardy_data.price.sort_values().unique()

array(['$1,000', '$1,020', '$1,100', '$1,111', '$1,183', '$1,200',
       '$1,203', '$1,246', '$1,263', '$1,300', '$1,347', '$1,400',
       '$1,407', '$1,492', '$1,500', '$1,512', '$1,534', '$1,600',
       '$1,700', '$1,777', '$1,800', '$1,801', '$1,809', '$1,810',
       '$1,900', '$10,000', '$10,400', '$10,800', '$100', '$1000',
       '$11,000', '$11,200', '$11,600', '$12,000', '$12,400', '$1200',
       '$13,000', '$13,200', '$13,800', '$14,000', '$14,200', '$16,400',
       '$1600', '$18,000', '$2,000', '$2,001', '$2,021', '$2,100',
       '$2,127', '$2,200', '$2,222', '$2,300', '$2,344', '$2,400',
       '$2,500', '$2,600', '$2,700', '$2,746', '$2,800', '$2,811',
       '$2,900', '$2,990', '$20', '$200', '$2000', '$22', '$250', '$2547',
       '$3,000', '$3,100', '$3,150', '$3,200', '$3,201', '$3,300',
       '$3,389', '$3,400', '$3,499', '$3,500', '$3,599', '$3,600',
       '$3,700', '$3,800', '$3,900', '$3,989', '$300', '$350', '$367',
       '$4,000', '$4,008', '$4,100', '$4

In [9]:
jeopardy_data.price.replace('\$(?P<dig1>\d+),?(?P<dig2>\d+)?$','\g<dig1>\g<dig2>', regex = True, inplace = True)
jeopardy_data.price.replace('None', 0, regex = True, inplace = True)

In [10]:
jeopardy_data.price = pd.to_numeric(jeopardy_data.price)
jeopardy_data.price.sort_values().unique()

array([    0,     5,    20,    22,    50,   100,   200,   250,   300,
         350,   367,   400,   500,   585,   600,   601,   700,   750,
         796,   800,   900,  1000,  1020,  1100,  1111,  1183,  1200,
        1203,  1246,  1263,  1300,  1347,  1400,  1407,  1492,  1500,
        1512,  1534,  1600,  1700,  1777,  1800,  1801,  1809,  1810,
        1900,  2000,  2001,  2021,  2100,  2127,  2200,  2222,  2300,
        2344,  2400,  2500,  2547,  2600,  2700,  2746,  2800,  2811,
        2900,  2990,  3000,  3100,  3150,  3200,  3201,  3300,  3389,
        3400,  3499,  3500,  3599,  3600,  3700,  3800,  3900,  3989,
        4000,  4008,  4100,  4200,  4238,  4300,  4400,  4500,  4600,
        4637,  4700,  4800,  5000,  5001,  5100,  5200,  5201,  5400,
        5401,  5500,  5600,  5700,  5800,  6000,  6100,  6200,  6300,
        6400,  6435,  6600,  6700,  6800,  7000,  7200,  7400,  7500,
        7600,  7800,  8000,  8200,  8400,  8500,  8600,  8700,  8800,
        8917,  9000,

### Clean `rounds` column
`rounds` only has five possible values. Convert to dtype "category".

In [11]:
jeopardy_data.rounds.unique()

array(['Jeopardy!', 'Double Jeopardy!', 'Final Jeopardy!', 'Tiebreaker'],
      dtype=object)

In [12]:
jeopardy_data['rounds'] = jeopardy_data['rounds'].astype("category")

### Clean `show_num` column
Read by pd.read_csv() as an integer. It takes a relatively high number of possible values => convert to object

In [13]:
jeopardy_data['show_num'] = jeopardy_data.show_num.astype("object")

In [14]:
jeopardy_data.show_num.describe()

count     216930
unique      3640
top         2941
freq          62
Name: show_num, dtype: int64

There are data from 3640 different shows of Jeopardy

### Clean `category` column
Leave it as object

In [15]:
jeopardy_data.category.describe()

count             216930
unique             27995
top       BEFORE & AFTER
freq                 547
Name: category, dtype: object

### Clean `air_date` column
It seems ok

In [16]:
jeopardy_data.air_date.describe(datetime_is_numeric=True)

count                           216930
mean     2003-03-08 00:06:26.734891648
min                1984-09-10 00:00:00
25%                1999-03-11 00:00:00
50%                2004-02-27 00:00:00
75%                2008-02-06 00:00:00
max                2012-01-27 00:00:00
Name: air_date, dtype: object

### Clean `answer` column
There are a couple of missing values in this column. `pd.read_csv()` loads 'null' as NaN by default. I am keeping it as default when loading and checking the NaN entries to confirm if they are proper `NaN`s (missing values) or if the answer in Jeopardy was 'null'. 

In [17]:
jeopardy_data.answer[jeopardy_data.answer.isnull()]

94817     NaN
143297    NaN
Name: answer, dtype: object

In [18]:
jeopardy_data.iloc[143297]

show_num                                                 6177
air_date                                  2011-06-21 00:00:00
rounds                                       Double Jeopardy!
category                                              NOTHING
price                                                     400
question    This word for "nothing" precedes "and void" to...
answer                                                    NaN
Name: 143297, dtype: object

In [19]:
jeopardy_data.iloc[94817]

show_num                            4346
air_date             2003-06-23 00:00:00
rounds                         Jeopardy!
category                   GOING "N"SANE
price                                200
question    It often precedes "and void"
answer                               NaN
Name: 94817, dtype: object

In both cases the answer was 'null', they are not really missing values.

In [20]:
jeopardy_data.answer.replace(np.nan, 'null', inplace= True)

In [21]:
jeopardy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   show_num  216930 non-null  object        
 1   air_date  216930 non-null  datetime64[ns]
 2   rounds    216930 non-null  category      
 3   category  216930 non-null  object        
 4   price     216930 non-null  int64         
 5   question  216930 non-null  object        
 6   answer    216930 non-null  object        
dtypes: category(1), datetime64[ns](1), int64(1), object(4)
memory usage: 10.1+ MB
